# II. Khám phá dữ liệu - EDA

Import các thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import datetime

/Users/huyton/opt/anaconda3/envs/min_ds-env/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Đọc dataset đã qua xử lý

In [2]:
df = pd.read_csv('Data/DelayedFlights_cleaned.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.head()

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,...,TaxiIn,TaxiOut,Cancelled,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Delayed,Timestamp
0,1,15,2,1948.0,19:05,2248.0,2217,XE,2597,N12996,...,9.0,11.0,0,31.0,0.0,0.0,0.0,0.0,True,2008-01-15 19:05:00
1,1,19,6,916.0,09:00,1204.0,1143,XE,1265,N14998,...,11.0,18.0,0,16.0,0.0,5.0,0.0,0.0,True,2008-01-19 09:00:00
2,1,20,7,1403.0,13:30,1625.0,1608,XE,1284,N16151,...,6.0,32.0,0,0.0,0.0,17.0,0.0,0.0,True,2008-01-20 13:30:00
3,1,31,4,1418.0,13:50,1810.0,1658,XE,2217,N29906,...,5.0,51.0,0,0.0,9.0,44.0,0.0,19.0,True,2008-01-31 13:50:00
4,1,2,3,1303.0,12:30,1557.0,1445,XE,2543,N16919,...,6.0,8.0,0,0.0,0.0,72.0,0.0,0.0,True,2008-01-02 12:30:00


---
## 1. Tìm hiểu về số lượng chuyến bay của các sân bay ở Georgia

In [3]:
top_10_ap = df['Origin'].value_counts().head(10)
px.bar(top_10_ap,
       x=top_10_ap.index,
       y=top_10_ap.values,
       title='Top 10 Airports with most flights',
       labels={'x':'Airport', 'y':'Number of Flights'},)

**Nhận xét**

- Nhìn chung, tại bang `Georgia` sân bay có lưu lượng bay cao nhất là `ATL`

- Số lượng bay tại sân bay này là cao nhất và có sự chênh lệch lớn giữa nó với phần còn lại

In [4]:
df['Dest'].value_counts()

Dest
ATL    71607
LGA     2961
SAV     2806
EWR     2779
ORD     2165
       ...  
ASE        7
MFE        3
FCA        3
CRP        1
GUC        1
Name: count, Length: 173, dtype: int64

----
## 2. Tìm hiểu về thời gian của các chuyến bay


Xét số lượng chuyến bay theo các thời điểm trong năm

In [5]:
flights_by_time = df['Timestamp'].dt.date.value_counts().sort_index()
px.line(flights_by_time,
        x=flights_by_time.index,
        y=flights_by_time.values,
        title='Number of Flights by Time',
        labels={'value':'Number of Flights', 'Timestamp':'Time'})

**Nhận xét**

- Số lượng chuyến bay trong năm có nhiều biến động

- Tuy nhiên, dường như dữ liệu cho thấy có tính chu kỳ

Xét theo tháng

In [6]:
top_by_month = df['Month'].value_counts().sort_index()
top_by_month.index=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

px.line(top_by_month,
         x=top_by_month.index,
         y=top_by_month.values,
         title='Number of Flights by Month',
         labels={'x':'Month', 'y':'Number of Flights'},
         markers=True)

**Nhận xét**

- Khi xét theo tháng, dễ dàng thấy được 3 tháng cao điểm rơi vào: T3, T7 và T12. Một vài lí do có thể như: T7 là rơi vào mùa hè nên nhu cầu đi lại và du lịch tăng cao, T12 có diễn ra lễ giáng sinh nên nhu cầu về nhà với gia đình cũng tăng và dây cũng là kì nghỉ khá dài.

- Ngoài các tháng cao điểm, số lượng chuyến bay có xu hướng giảm, thấp nhất vào các tháng: T5, T9

- **Tính chu kỳ**: có thể thấy rằng số lượng chuyến bay thường đạt đỉnh theo chu kỳ khoảng 5 tháng.

    - Ví dụ: một chu kỳ T1-T5

Xét theo các ngày trong tuần

In [7]:
top_by_weekday = df['DayOfWeek'].value_counts().sort_index()
top_by_weekday.index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

px.bar(top_by_weekday,
       x=top_by_weekday.index,
       y=top_by_weekday.values,
       title='Number of Flights by Day of the Week',
       labels={'y':'Number of Flights', 'x':'Day of the Week',})


**Nhận xét**

- Thời gian các chuyến bay diễn ra phân bố tương đối đều nhau giữa các ngày trong tuần

- Thời gian cao điểm thường rơi vào ngày thứ 6

- Lý do T6 khá cao có thể do người ta đi làm ở các thành phố hoặc bang khác, nên ngày T6 (ngày làm cuối cùng) người ta sẽ bay về nhà của mình

- Ngày T2 và ngày CN cũng khá cao, lý do có thể là vì sau khi họ về nhà thì phải trở lại làm việc vào tuần mới nên số chuyến bay có thể tăng lên theo.

Xét theo giờ

In [8]:
flight_hours = df['Timestamp'].dt.hour.value_counts().sort_index()
px.line(flight_hours,
       x=flight_hours.index,
       y=flight_hours.values,
       title='Number of Flights by Hour',
       labels={'x':'Hour', 'y':'Number of Flights'})

**Nhận xét**

- Số lượng chuyến bay có xu hướng tăng từ sau 5h sáng

- Đạt đỉnh điểm tại lúc 16h và có xu hướng giảm dần ngay sau đó

- Từ sau 23h, hầu như số lượng chuyến bay là rất ít

----
## 3. Tìm hiểu về các chuyến bay bị delayed

Xem phân bố của thời gian delay khi khởi hành

In [9]:
px.histogram(df,
             x='DepDelay',
             title='Departure Delay Distribution',
             labels={'DepDelay':'Departure Delay (minutes)'})


**Nhận xét**

- Thời gian delay chủ yếu là ở mức 20p

In [10]:
delayed_flights = df[df['Delayed']  == True]

In [11]:
date_counts = delayed_flights['Timestamp'].dt.date.value_counts().sort_index()
px.line(date_counts,
        x=date_counts.index,
        y=date_counts.values,
        title='Number of Delayed Flights by Date',
        labels={'x':'Date', 'y':'Number of Delayed Flights'})

**Nhận xét**

- Sự biến đổi của dữ liệu tương tự như số lượng chuyến bay

- Dữ liệu cũng cho thấy có tính chu kỳ

- Có nhiều sự biến động trong dữ liệu

Xét số lượng chuyến bay bị delayed theo tháng

In [12]:
delayed_flights_by_month = delayed_flights['Month'].value_counts().sort_index()
px.line(delayed_flights_by_month,
            x=delayed_flights_by_month.index,
            y=delayed_flights_by_month.values,
            title='Number of Delayed Flights by Month',
            labels={'x':'Month', 'y':'Number of Delayed Flights'},
            markers=True)

**Nhận xét**

- Số lượng chuyến bay bị delayed theo tháng có xu hướng tương tự với số lượng chuyến bay

- **Tính chu kỳ**: tương tự như tính chu kỳ của số lượng chuyến bay. Chu kỳ cũng có kéo dài khoảng 5 tháng. Bởi vì số lượng chuyến bay càng nhiều thì số lượng bị delayed sẽ tăng theo tương ứng.

Xét theo các ngày trong tuần

In [13]:
weekday_counts = delayed_flights['DayOfWeek'].value_counts().sort_index()
weekday_counts.index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

px.bar(weekday_counts,
         x=weekday_counts.index,
         y=weekday_counts.values,
         title='Number of Delayed Flights by Day of the Week',
         labels={'x':'Day of the Week', 'y':'Number of Delayed Flights'})

**Nhận xét**

- Ngày T6 là có số chuyến bay delayed cao nhất. Có thể đây là ngày cuối tuần, nhiều người cần di chuyển về gia đình... nên số chuyến bay tăng cao dẫn đến tình trạng delay diễn ra nhiều

Xét theo giờ

In [14]:
hour_counts = delayed_flights['Timestamp'].dt.hour.value_counts().sort_index()
px.line(hour_counts,
         x=hour_counts.index,
         y=hour_counts.values,
         title='Number of Delayed Flights by Hour',
         labels={'x':'Hour', 'y':'Number of Delayed Flights'})

**Nhận xét**

- Xu hướng của thời gian diễn ra delay tương đồng với xu hướng của số lượng chuyến bay

- Tại các mốc thời gian từ 12h - 21h, xảy ra rất nhiều vụ delay

In [15]:
top_10_delayed_ap = delayed_flights.groupby('Origin')['Origin'].value_counts().sort_values(ascending=False).head(10)
px.bar(top_10_delayed_ap,
       x=top_10_delayed_ap.index,
       y=top_10_delayed_ap.values,
       title='Top 10 Airports with most delayed flights',
       labels={'x':'Airport', 'y':'Number of Delayed Flights'})

- ATL là sân bay có lưu lượng bay cao nhất. Do đó cũng dễ hiểu rằng số lượng chuyến bay bị delayed của nó cũng là cao nhất

In [16]:
delay_factors = delayed_flights[['CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay']].sum().sort_values(ascending=False)

px.pie(delay_factors,
       values=delay_factors.values,
       names=delay_factors.index,
       title='Delay Factors',
       labels={'value':'Number of Minutes', 'names':'Delay Type'})

**Nhận xét**

- Nhìn chung, 2 yếu tố chính chiếm phần lớn thời gian gây delay đó chính là: do hãng hàng không và do sự chậm trễ của các chuyến bay trước đó gây ra